In [1]:
# la fuente del codigo es: https://github.com/sankit1/cv-tricks.com

#importa la clase conjuntoDeDatos, esta obtiene las imagenes de entrenamiento por cada subcarpeta o clases de imagenes. 
import CNN
import conjuntoDeDatos
import tensorflow as tf
from datetime import timedelta
import random
import os
import math
import time
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# instanciar CNN
cnncapas = CNN.CNN()

# Agregar semilla para que la inicialización aleatoria sea consistente
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

""" tamanoLotePorIteracion (batch-size) """
tamanoLotePorIteracion = 32

#Preparar la data de entrada
#Existen dos calses, perros y gatos.
clases = ['dog','cat']
numeroClases = len(clases)

# leendo las imagenes de entrada
# datos de entrenamiento: deberia ser usado el 80% de las imagenes
# datos de validacion: deberia ser usado el 20% de las imagenes, estas imagenes deben quedar fuera del conjunto de datos de entrenamiento, este conjunto de datos es necesario para calcular la exactitud del modelo.
# datos de prueba, este conjunto de datos es utilizado para probar el modelo, despue de haber entrenado el modelo funciona bien, pero cuando las imagenes son convertidas a un tamaño muy reducido el model puede fallar, a este termino se le conoce como sobre sobreajuste(Overfitting), el sobre ajuste puede ser probocado por el fondo de las imagenes.

# tamanoDeDataDeValidacion automaticamente se usara el 20% de todas las imagenes para la validacion.
tamanoDeDataDeValidacion = 0.2

# El mamaño de las imagenes es de 128 pixeles por 128.
tamanoDeImagenes = 128

# el numero de canales representa que la imagenen tiene los tres colores RGB
numeroDeColoresPorImagen = 3

# la ruta de datos de entrenamiento es la carpeta donde se encuentran los conjuntos de imagenes de entrenamiento y validacion
rutaDeDatosDeEntrenamiento=os.path.join(os.path.realpath('.'),'imagenes/train/')
print("ruta de datos de entrenamiento "+str(rutaDeDatosDeEntrenamiento))
# Cargar las imagenes de entrenamiento y validacion con sus etiquetas en memoria usando openCV para ser usadas durante el proceso de entrenamiento.
data = conjuntoDeDatos.leerDatosDeEntrenamiento(rutaDeDatosDeEntrenamiento, tamanoDeImagenes, clases, tamanoDeDataDeValidacion)

print("Completado el proceso de lecturada de imagenes.")
print("Numero de imagenes en el conjunto de entrenamiento:\t{}".format(len(data.entrenamiento.etiquetas)))
print("Numero de imagenes en el conjunto de validacion :\t{}".format(len(data.validacion.etiquetas)))

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ruta de datos de entrenamiento /notebooks/CNNcatdogs/imagenes/train/
clase dog, indice 0, path /notebooks/CNNcatdogs/imagenes/train/dog.*.jpg, files number 12243, max images 12243
progress, read 999 of 12243
progress, read 1999 of 12243
progress, read 2999 of 12243
progress, read 3999 of 12243
progress, read 4999 of 12243
progress, read 5999 of 12243
progress, read 6999 of 12243
progress, read 7999 of 12243
progress, read 8999 of 12243
progress, read 9999 of 12243
progress, read 10999 of 12243
progress, read 11999 of 12243
clase cat, indice 1, path /notebooks/CNNcatdogs/imagenes/train/cat.*.jpg, files number 12500, max images 12500
progress, read 999 of 12500
progress, read 1999 of 12500
progress, read 2999 of 12500
progress, read 3999 of 12500
progress, read 4999 of 12500
progress, read 5999 of 12500
progress, read 6999 of 12500
progress, read 7999 of 12500
progress, read 8999 of 12500
progress, read 9999 of 12500
progress, read 10999 of 12500
progress, read 11999 of 12500
End to get 

In [2]:
session = tf.Session()
tensorDeEntrada = tf.placeholder(tf.float32, shape=[None, tamanoDeImagenes,tamanoDeImagenes,numeroDeColoresPorImagen], name='tensorDeEntrada')

## Etiquetas
tensorDeClases = tf.placeholder(tf.float32, shape=[None, numeroClases], name='tensorDeClases')
tensorDeClasesAplanado = tf.argmax(tensorDeClases, dimension=1)

In [3]:
##Parametros graficos de red
# El parametro mas importante en una red convolucional es el tamaño del filtro de cada neurona.  Por ejemplo si entrada de la neurona es una imagen de 32x32x3, esto significa que la imagen tiene una dimension de 32x32 (alto x ancho) pixeles y contiene 3 colores RGB, entonces el filtro significa que se convertiran las imagenes a 5x5x3, es decir 5x5 (alto x ancho) y 3 representa el numero de colores RGB.

#Configuracion de la primera capa oculta, el tamaño del filtro es 3 y el numero de filtros es 32

#capaConvolucional1 = cnncapas.(tensorDeEntrada,3,3,32)
capaConvolucional1 = cnncapas.crearCapaConvolucional(
                        tensorDeEntrada=tensorDeEntrada,
                        numeroDeCanales=numeroDeColoresPorImagen,
                        tamanoDelFiltro=3,
                        numeroDeFiltros=32)

#capaConvolucional2 = (capaConvolucional1,32,3,32)
capaConvolucional2 = cnncapas.crearCapaConvolucional(
                        tensorDeEntrada=capaConvolucional1,
                        numeroDeCanales=32,
                        tamanoDelFiltro=3,
                        numeroDeFiltros=32)

#capaConvolucional3= cnncapas.(capaConvolucional2,32,3,64)
capaConvolucional3= cnncapas.crearCapaConvolucional(
                        tensorDeEntrada=capaConvolucional2,
                        numeroDeCanales=32,
                        tamanoDelFiltro=3,
                        numeroDeFiltros=64)

capaPlana = cnncapas.crearCapaAplanada(capaConvolucional3)

capaTotalmenteConectada1 = cnncapas.crearCapaTotalmenteConectada(
                            tensorDeEntrada=capaPlana,
                            num_inputs=capaPlana.get_shape()[1:4].num_elements(),
                            num_outputs=128,
                            use_relu=True)

capaTotalmenteConectada2 = cnncapas.crearCapaTotalmenteConectada(
                            tensorDeEntrada=capaTotalmenteConectada1,
                            num_inputs=128,
                            num_outputs=numeroClases,
                            use_relu=False)

In [4]:
#Softmax: Calcula las activaciones softmax, los argumentos de la funcion son:
## logits: es un tenson no vacio.  Puede ser uno de los siguientes tipos "half, float32, float64"
## dim: la dimension softmax puede ser realidad en: el valor default es -1, el cual indica la ultima dimension.
## name: es opcional, representa el nombre de la operacion.

## retorna: un tensor, tiene el mismo tipo y forma que logits.

# la variable ResultadoDeProbabilidadPorClase guarda los valores de la ultima capa (capaTotalmenteConectada2) y obtiene por medio de la funcion softmax los valores de probabilidad en un rango de 0-1
prediccionDeProbabilidadPorClase = tf.nn.softmax(
                    capaTotalmenteConectada2,
                    name='prediccionDeProbabilidadPorClase')

# la variable ResultadoDeProbabilidadPorClase guarda la probabilidad de cada clase aplanda es decir en un vector.
prediccionDeProbabilidadPorClaseAplanada = tf.argmax(
                    prediccionDeProbabilidadPorClase, 
                    dimension=1)

# tf.Session es una clase para correr operaciones de tensorflow
# Un objetivo de una sesion es encapsular el entorno en el cual los objectos (operaciones) son ejecutados y los objectos (Tensores) son evaluados.

#inicializar las variables.
session.run(tf.global_variables_initializer())

#Entropia cruzada:
entropiaCruzada = tf.nn.softmax_cross_entropy_with_logits(
                    logits=capaTotalmenteConectada2,
                    labels=tensorDeClases)

perdidaTotalDeErrorEnClasificacion = tf.reduce_mean(entropiaCruzada)

optimizador = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(perdidaTotalDeErrorEnClasificacion)

correccionDePrediccion = tf.equal(prediccionDeProbabilidadPorClaseAplanada, 
                                  tensorDeClasesAplanado)

exactitudDelModelo = tf.reduce_mean(tf.cast(correccionDePrediccion, tf.float32))

#inicializar las variables.
tf.summary.FileWriter("/tmp/tensorflow/", session.graph)
session.run(tf.global_variables_initializer())


In [5]:
def mostrarProgreso(epoca, entrenamiento_imagenesYetiquetasPorEpoca, validacion_imagenesYetiquetasPorEpoca, perdidaDeValidacion):
    exactitudDeEntrenamiento = session.run(exactitudDelModelo, feed_dict=entrenamiento_imagenesYetiquetasPorEpoca)
    exactitudDeValidacion = session.run(exactitudDelModelo, feed_dict=validacion_imagenesYetiquetasPorEpoca)
    msg = "Training epoca {0} --- Exactitud del entranamiento: {1:>6.1%}, Exactitud de validacion: {2:>6.1%},  perida de validacion: {3:.3f}"
    print(msg.format(epoca + 1, exactitudDeEntrenamiento, exactitudDeValidacion, perdidaDeValidacion))

recuentoIteraciones = 0

""" batch-size (tamaño del lote por iteracion), cuando se entrena una red no se  alimenta con todo el conjunto de datos, se hace por medio de iteracion y cada iteracion tiene un tamaño, frecuentemente es 16 o 32"""
saver = tf.train.Saver()

def entrenarModelo(totalIteraciones):
    print("---------------- FUNCION DE ENTRENAMIENTO  ------------------")
    print("numero de imagenes para entrenamiento "+str(data.entrenamiento.recuento))

    global recuentoIteraciones
    for iteracion in range(recuentoIteraciones,recuentoIteraciones + totalIteraciones):
        if((iteracion+1) % 100 == 0):
            print("-- iteracion # "+str(iteracion+1)+" con imagenes # "+str(tamanoLotePorIteracion))
        
        entrenamientoPaquete_imagenes, entrenamientoPaquete_etiquetas, entrenamientoPaquete_nombres, entrenamientoPaquete_clases = data.entrenamiento.siguienteLote(tamanoLotePorIteracion)
        validacionPaquete_imagenes, validacionPaquete_etiquetas, validacionPaquete_nombres, validacionPaquete_clases = data.validacion.siguienteLote(tamanoLotePorIteracion)

        entrenamiento_imagenesYetiquetasPorEpoca = {tensorDeEntrada: entrenamientoPaquete_imagenes,tensorDeClases: entrenamientoPaquete_etiquetas}
        validacion_imagenesYetiquetasPorEpoca = {tensorDeEntrada: validacionPaquete_imagenes,tensorDeClases: validacionPaquete_etiquetas}
        
        session.run(optimizador, feed_dict=entrenamiento_imagenesYetiquetasPorEpoca)

        if iteracion % int(data.entrenamiento.recuento/tamanoLotePorIteracion) == 0:
            print("mostrar epoca "+str(int(data.entrenamiento.recuento/tamanoLotePorIteracion)))
            perdidadDeValidacion = session.run(perdidaTotalDeErrorEnClasificacion, feed_dict=validacion_imagenesYetiquetasPorEpoca)

            """ epoca (epoch) una epoca contiene un conjunto de iteracion que alimentan el modelo """
            epoca = int(iteracion / int(data.entrenamiento.recuento/tamanoLotePorIteracion))

            mostrarProgreso(epoca, entrenamiento_imagenesYetiquetasPorEpoca, validacion_imagenesYetiquetasPorEpoca, perdidadDeValidacion)
            saver.save(session, './model/codigoModificado/dogs-cats-model_20_04_2018_v.1.meta')

    recuentoIteraciones += totalIteraciones

entrenarModelo(totalIteraciones=4000)
print("Fin del entrenamiento")

---------------- FUNCION DE ENTRENAMIENTO  ------------------
numero de imagenes para entrenamiento 19794
mostrar epoca 618
Training epoca 1 --- Exactitud del entranamiento:  53.1%, Exactitud de validacion:  59.4%,  perida de validacion: 0.691
-- iteracion # 100 con imagenes # 32
-- iteracion # 200 con imagenes # 32
-- iteracion # 300 con imagenes # 32
-- iteracion # 400 con imagenes # 32
-- iteracion # 500 con imagenes # 32
-- iteracion # 600 con imagenes # 32
mostrar epoca 618
Training epoca 2 --- Exactitud del entranamiento:  59.4%, Exactitud de validacion:  65.6%,  perida de validacion: 0.645
-- iteracion # 700 con imagenes # 32
-- iteracion # 800 con imagenes # 32
-- iteracion # 900 con imagenes # 32
-- iteracion # 1000 con imagenes # 32
-- iteracion # 1100 con imagenes # 32
-- iteracion # 1200 con imagenes # 32
mostrar epoca 618
Training epoca 3 --- Exactitud del entranamiento:  59.4%, Exactitud de validacion:  81.2%,  perida de validacion: 0.408
-- iteracion # 1300 con imagenes 